In [1]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re

In this notebook, we will get:
- Dataframes of tweets of each president of the party. 
- Dataframes of mentions of users for each president.

### Connecting with Twitter API

In [2]:
%run ./keys.ipynb
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# General Functions

In [3]:
def extracting_mentions(query):
    """
    This function extract a data frame with the result of a query.
    """
    mentions = [tweet for tweet in tweepy.Cursor(api.search, 
                                                q=query + ' -filter:retweets', 
                                                lang="es", 
                                                tweet_mode='extended',
                                                result_type="recent").items(100)]
    mentions_json = [tweet._json for tweet in mentions]
    df_mentions = pd.json_normalize(mentions_json)
    
    # selecting useful columns
    columns_selected = ['user.name', 'created_at', 'id', 'full_text', 'display_text_range', 
                    'source', 'retweet_count', 'favorite_count', 'user.followers_count', 
                    'user.friends_count', 'user.statuses_count', 'entities.hashtags', 'entities.urls']
    
    df_mentions_filtered = df_mentions[columns_selected]
    
    # cleaning date time
    df_mentions_filtered['created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
    
    # cleaning source of the tweet
    list_sources = list(df_mentions_filtered['source'])
    df_mentions_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]
    
    return df_mentions_filtered

In [4]:
def adding_party_column(df, string):
    df['partido'] = string
    return df

In [5]:
def adding_person_column(df, string):
    df['persona'] = string
    return df

In [6]:
def adding_type_post_column(df, string):
    """
    string = publicación o mención
    """
    df['tipo de post'] = string
    return df

In [7]:
def transforming_format_dates(column):
    return pd.to_datetime(column)

In [8]:
columns_selected = ['user.name', 'created_at', 'id', 'full_text', 'display_text_range', 
                    'source', 'retweet_count', 'favorite_count', 'user.followers_count', 
                    'user.friends_count', 'user.statuses_count', 'entities.hashtags', 'entities.urls']

## 1. PARTIDO POPULAR

In [9]:
pp_query = '@populares OR "partido popular" OR "el pp" OR "los del pp" -filter:retweets'

In [10]:
df_pp_mentions = extracting_mentions(pp_query)

<ipython-input-3-3e0d8a736f17>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
<ipython-input-3-3e0d8a736f17>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [11]:
df_pp_mentions = adding_party_column(df_pp_mentions, 'Partido Popular')
df_pp_mentions = adding_type_post_column(df_pp_mentions, 'mención')

In [12]:
df_pp_mentions

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,partido,tipo de post
0,Pilar,2021-04-17 11:05:55+00:00,1383376168754180096,@InesArrimadas Más que tú ???? El PP os ha tom...,"[15, 163]",[Twitter for iPhone],0,0,2641,2920,21584,[],[],Partido Popular,mención
1,Sergio R.Aranoa 🌹🔻 #GabilondoPresidente,2021-04-17 11:05:45+00:00,1383376129654812672,@ppmadrid @populares Con sobrecostes de 100 mi...,"[21, 88]",[Twitter for iPhone],0,0,3617,4997,151810,[],[],Partido Popular,mención
2,Toni🌾🥀,2021-04-17 11:05:44+00:00,1383376124365840385,Ventajas de esta estrategia:\n\n- los votantes...,"[0, 237]",[Twitter Web App],0,0,142,185,5079,[],[],Partido Popular,mención
3,Ángel Panizo Coleto,2021-04-17 11:05:42+00:00,1383376116166004747,@populares @IdiazAyuso Aqui están los datos de...,"[23, 300]",[Twitter for Android],0,0,433,127,4316,[],"[{'url': 'https://t.co/x1MBawrDo7', 'expanded_...",Partido Popular,mención
4,Salome,2021-04-17 11:05:39+00:00,1383376102983299076,@teresajbecerril @populares También hablabas a...,"[28, 104]",[Twitter for iPhone],0,0,200,381,3553,[],[],Partido Popular,mención
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Isabel Dáz Ayuso ᶠᵃᵏᵉ,2021-04-17 10:52:55+00:00,1383372899352866818,"@populares @IdiazAyuso Yo tenía la ""Operación ...","[23, 67]",[Twitter Web App],0,0,440,878,1353,[],[],Partido Popular,mención
96,Eduardo Loren García,2021-04-17 10:52:52+00:00,1383372883888459777,"🗳️ ""Madrid corre hoy el riesgo de convertirse ...","[0, 280]",[Twitter Web App],1,2,3985,2549,84660,[],"[{'url': 'https://t.co/0WSoBXoTBv', 'expanded_...",Partido Popular,mención
97,Residents Corner,2021-04-17 10:52:37+00:00,1383372823209472003,Necesitamos un gob decente en Madrid. En los 2...,"[0, 279]",[Twitter for Android],0,0,4066,4382,16312,[],[],Partido Popular,mención
98,Jorge Zuazola,2021-04-17 10:52:29+00:00,1383372788237361155,@PalmeiroRicardo @carlosdavidgf @Tonicanto1 @S...,"[197, 239]",[Twitter Web App],0,0,1705,4909,147048,[],[],Partido Popular,mención


## 2. PSOE

In [13]:
psoe_query ='@psoe OR "partido socialista" OR "Partido Socialista Obrero Español" OR "el psoe" OR "los del psoe" -filter:retweets'

In [14]:
df_psoe_mentions = extracting_mentions(psoe_query)

<ipython-input-3-3e0d8a736f17>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
<ipython-input-3-3e0d8a736f17>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [15]:
adding_party_column(df_psoe_mentions, 'PSOE')
adding_type_post_column(df_psoe_mentions, 'mención')

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,partido,tipo de post
0,kiyo que? ۞,2021-04-17 11:06:11+00:00,1383376236513169409,@rosadiezglez @PSOE @PODEMOS @MasMadridCM Los ...,"[42, 211]",[Twitter for iPhone],0,0,54,224,1832,[],[],PSOE,mención
1,Jose García Buitrón,2021-04-17 11:05:28+00:00,1383376057269592070,"Si en Madrid no gana PP/VOX, Gabilondo abrirá ...","[0, 278]",[Twitter Web App],0,1,23157,20950,42288,[],[],PSOE,mención
2,the punisher,2021-04-17 11:05:26+00:00,1383376050294456320,@PSOE @equipoGabilondo Para estos el progresis...,"[23, 187]",[Twitter for iPhone],0,0,167,183,35548,[],[],PSOE,mención
3,Alfonso Cordero,2021-04-17 11:05:15+00:00,1383376001330155520,"@PSOE @equipoGabilondo Por favor, dejen de hac...","[23, 295]",[Twitter Web App],0,0,212,222,9548,[],"[{'url': 'https://t.co/7s0lkX4LAU', 'expanded_...",PSOE,mención
4,R. Martínez,2021-04-17 11:05:12+00:00,1383375989078511625,@telephoneOD @HablamosE @Theo_Anrochte\nPP tam...,"[13, 292]",[Twitter for Android],0,0,56,25,3363,[],[],PSOE,mención
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,galivor,2021-04-17 10:49:54+00:00,1383372141244993542,@Emiliovk El PSOE es una franquicias de la der...,"[10, 50]",[Twitter Web App],0,0,18,65,692,[],[],PSOE,mención
96,Duimovich 🔱,2021-04-17 10:49:43+00:00,1383372093635448841,"Los ricos estarán temblando, menos mal que el ...","[0, 123]",[Twitter for iPhone],0,0,66,78,4166,[],"[{'url': 'https://t.co/Kpz580mLJi', 'expanded_...",PSOE,mención
97,Estrella Maimónides,2021-04-17 10:49:42+00:00,1383372089298587650,@juanjerez54 @rosadiezglez @PSOE @PODEMOS @Mas...,"[55, 260]",[Twitter for Android],0,1,73,589,5276,[],[],PSOE,mención
98,kr4M3r,2021-04-17 10:49:38+00:00,1383372073318244353,@TvPlataforma @rtve @PSOE @telediario_tve Panf...,"[42, 72]",[Twitter for iPhone],0,0,37,293,1142,[],[],PSOE,mención


## 3. CIUDADANOS

In [16]:
ciudadanos_query = '@CiudadanosCs OR "ciudadanos" OR "los de ciudadanos" -filter:retweets'

In [17]:
df_ciudadanos_mentions = extracting_mentions(ciudadanos_query)

<ipython-input-3-3e0d8a736f17>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
<ipython-input-3-3e0d8a736f17>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [18]:
adding_type_post_column(df_ciudadanos_mentions, 'mención')
adding_party_column(df_ciudadanos_mentions, 'Ciudadanos')

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,tipo de post,partido
0,HERNANDO GUERRERO VALENCIA,2021-04-17 11:06:20+00:00,1383376273683058696,@NatalyCortezG @Orlando71156528 @sergiobarbosa...,"[49, 296]",[Twitter for Android],0,0,116,1449,74,[],[],mención,Ciudadanos
1,Álvaro Fergez,2021-04-17 11:06:15+00:00,1383376253927911428,@MythicalIberia @Piazza_Paya2 Podían irse todo...,"[30, 144]",[Twitter for Android],0,0,2739,4565,7952,[],[],mención,Ciudadanos
2,PatosPP,2021-04-17 11:06:06+00:00,1383376217483603979,@MartinSiracusa Cuando las fuerzas federales c...,"[16, 98]",[Twitter for iPhone],0,0,141,158,14371,[],[],mención,Ciudadanos
3,Martin Cáceres 🇵🇪🏚📖🎸,2021-04-17 11:06:03+00:00,1383376203461976072,@elcomercio_peru Con seguridad lo será; parali...,"[17, 116]",[Twitter for Android],0,0,431,1162,26034,[],[],mención,Ciudadanos
4,Damian C,2021-04-17 11:05:52+00:00,1383376156351598598,@pabloelleon1 @waledi52 @1080marcelo @mariuch3...,"[70, 349]",[Twitter Web App],0,0,70,264,1055,"[{'text': 'presidenteausente', 'indices': [331...",[],mención,Ciudadanos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Ángel Bravo,2021-04-17 10:50:41+00:00,1383372335961440263,@militaresantifa @micro16f84a Tendríamos que t...,"[30, 304]",[Twitter Web App],0,1,1962,5001,224166,[],[],mención,Ciudadanos
96,《《Maribel Orellana》》#AAFFinkieta,2021-04-17 10:50:35+00:00,1383372313249210368,Los intereses políticos y los privados de las ...,"[0, 217]",[Twitter for Android],0,0,3744,1525,87850,"[{'text': 'AsíNo', 'indices': [174, 180]}]","[{'url': 'https://t.co/80dwZoUCz0', 'expanded_...",mención,Ciudadanos
97,Rubén Valverde,2021-04-17 10:50:29+00:00,1383372285420077056,@Isabelgranadina @navedelmisterio Bueno la inf...,"[34, 313]",[Twitter for iPhone],0,0,6578,5972,4118,[],[],mención,Ciudadanos
98,Rocío Fernández 🖤 🇪🇸,2021-04-17 10:50:27+00:00,1383372277924843523,@CsMJe @Cs_Madrid @CiudadanosCs En Madrid @Cs_...,"[32, 231]",[Twitter for Android],3,1,631,1018,6669,"[{'text': 'Selloanaranja', 'indices': [97, 111...",[],mención,Ciudadanos


## 4. PODEMOS

In [19]:
podemos_query = '@PODEMOS OR "podemos" OR "los de podemos" OR "podemitas" OR "el psoe" OR "el de podemos" -filter:retweets'

In [20]:
df_podemos_mentions = extracting_mentions(podemos_query)

<ipython-input-3-3e0d8a736f17>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
<ipython-input-3-3e0d8a736f17>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [21]:
adding_type_post_column(df_podemos_mentions, 'mención')
adding_party_column(df_podemos_mentions, 'PODEMOS')

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,tipo de post,partido
0,Luisma Casado Gamero,2021-04-17 11:06:48+00:00,1383376392335740933,Quién está empeñado en difundir que el PSOE ...,"[0, 199]",[Twitter for Android],0,0,165,1041,3911,[],[],mención,PODEMOS
1,Gaijin,2021-04-17 11:06:46+00:00,1383376383938727943,Podemos y eso... https://t.co/7FLskwZF8x,"[0, 16]",[Twitter for Android],0,0,85,550,9424,[],[],mención,PODEMOS
2,𝓁𝒾𝓃𝒶 王一博,2021-04-17 11:06:43+00:00,1383376371221598216,@splengguk Sí podemos!!!!,"[11, 25]",[Twitter for Android],0,0,558,632,28457,[],[],mención,PODEMOS
3,"Juanillo #yoconpabloiglesias, Ya en ⭕️ de Pode...",2021-04-17 11:06:39+00:00,1383376354591219712,"Salgan de sus casas, cada persona un voto.\n\n...","[0, 136]",[Twitter Web App],0,0,855,1320,50044,"[{'text': 'QueHableLaMayoria', 'indices': [54,...","[{'url': 'https://t.co/eykdDSN0xu', 'expanded_...",mención,PODEMOS
4,Julia ∞,2021-04-17 11:06:38+00:00,1383376352150130695,@Pep_Rincon @PabloEchenique En Podemos se hace...,"[28, 173]",[Twitter for Android],0,0,10897,9447,153570,[],[],mención,PODEMOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Juanillo #yoconpabloiglesias, Ya en ⭕️ de Pode...",2021-04-17 11:02:58+00:00,1383375429411631107,"Salgan de sus casas, cada persona un voto.\n\n...","[0, 136]",[Twitter Web App],0,0,855,1320,50044,"[{'text': 'QueHableLaMayoria', 'indices': [54,...","[{'url': 'https://t.co/ZUtilAPZwt', 'expanded_...",mención,PODEMOS
96,Biblioteca IES LA FUENSANTA. Fuenteca.,2021-04-17 11:02:54+00:00,1383375411359326214,"Nuestro querido compañero, @Nando_Lopez_ ha re...","[0, 219]",[Twitter Web App],1,1,53,74,108,[],"[{'url': 'https://t.co/5sXDeZW5aN', 'expanded_...",mención,PODEMOS
97,Pablo Beltrán-Pellicer 🧮🍏,2021-04-17 11:02:53+00:00,1383375407139852290,"@p_trivino Simplemente digo que no basta. Cid,...","[11, 288]",[Twitter for Android],0,0,6360,2224,19576,[],[],mención,PODEMOS
98,Quilmes Silvia,2021-04-17 11:02:52+00:00,1383375402333208581,Muy triste \n\nAquí podemos parar la pelota ya...,"[0, 222]",[Twitter for Android],0,0,1781,1892,34794,"[{'text': 'CuidarteEsCuidarnos', 'indices': [2...","[{'url': 'https://t.co/dUUArWgHS0', 'expanded_...",mención,PODEMOS


## 5. VOX - Santiago Abascal

In [22]:
vox_query = '@vox_es OR "vox" OR "los de vox" OR "el de VOX -filter:retweets"'

In [23]:
df_vox_mentions = extracting_mentions(vox_query)

<ipython-input-3-3e0d8a736f17>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['created_at'] = pd.to_datetime(df_mentions_filtered['created_at'])
<ipython-input-3-3e0d8a736f17>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mentions_filtered['source'] = [re.findall(r'\>(.*?)\<', s) for s in list_sources]


In [24]:
adding_type_post_column(df_vox_mentions, 'mención')
adding_party_column(df_vox_mentions, 'Vox')

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,tipo de post,partido
0,ALBERTO 🇪🇸,2021-04-17 11:07:07+00:00,1383376471863939073,@madrid_vox @Macarena_Olona Allí estuve!! Eso ...,"[28, 84]",[Twitter for Android],0,0,1859,3814,4896,[],[],mención,Vox
1,Jordi Meneses,2021-04-17 11:07:04+00:00,1383376458500829188,"Y de esto trata la política en general, gente ...","[0, 279]",[Twitter for Android],0,0,48,587,2039,[],[],mención,Vox
2,LeoNina 🇪🇸👑💚,2021-04-17 11:07:02+00:00,1383376452637233154,Otra represión policial en la Habana esto es e...,"[0, 273]",[Twitter for Android],0,0,1028,1180,1244,"[{'text': 'PrimeroVox', 'indices': [224, 235]}...","[{'url': 'https://t.co/gqYhX9pF4f', 'expanded_...",mención,Vox
3,SpanishKhan🔻✊💜🌹🇵🇸🇪🇺,2021-04-17 11:06:59+00:00,1383376439584514048,"@InesArrimadas El plan mecuida, la equiparació...","[15, 274]",[Twitter for Android],0,0,126,893,3820,[],[],mención,Vox
4,LeonelSanchezCamero 🇪🇸🇻🇪🇺🇲🇪🇺,2021-04-17 11:06:57+00:00,1383376432022245381,@chispigordi @TheCynicalHun @Sunshin29891796 @...,"[273, 280]",[Twitter for Android],0,0,4645,3876,5142,[],[],mención,Vox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Vox_Hellin,2021-04-17 10:58:55+00:00,1383374409545576455,‼️Nosotros luchamos por todos los españoles si...,"[0, 158]",[Twitter for Android],0,0,290,338,3340,"[{'text': 'ProtegeMadrid', 'indices': [125, 13...",[],mención,Vox
96,Francisco J.rey /🌹❤,2021-04-17 10:58:49+00:00,1383374384346267651,@Sirona33014070 @TaniaCrespo3 @LorenaCastro20 ...,"[60, 198]",[Twitter for Android],0,0,1005,991,5732,[],[],mención,Vox
97,christian9516,2021-04-17 10:58:40+00:00,1383374347344125957,@TabarniaB Sería noticia que Vox no llenará el...,"[11, 63]",[Twitter for Android],0,0,23,75,2859,[],[],mención,Vox
98,justi,2021-04-17 10:58:21+00:00,1383374264561061891,El doble pinchazo de Vox en Orcasitas: ni públ...,"[0, 87]",[Twitter for iPhone],0,0,954,976,81113,[],"[{'url': 'https://t.co/aNGIIqo431', 'expanded_...",mención,Vox


## Data frame final mentions

In [25]:
frames = [df_pp_mentions, df_psoe_mentions, df_ciudadanos_mentions, df_podemos_mentions, df_vox_mentions]
df_final_mentions = pd.concat(frames)

In [29]:
# exporting
df_final_mentions.to_csv('../data/processed/df_final_mentions_2.csv')

In [28]:
df_final_mentions.shape

(500, 15)

In [30]:
df_final_mentions

,user.name,created_at,id,full_text,display_text_range,source,retweet_count,favorite_count,user.followers_count,user.friends_count,user.statuses_count,entities.hashtags,entities.urls,partido,tipo de post
0,Pilar,2021-04-17 11:05:55+00:00,1383376168754180096,@InesArrimadas Más que tú ???? El PP os ha tom...,"[15, 163]",[Twitter for iPhone],0,0,2641,2920,21584,[],[],Partido Popular,mención
1,Sergio R.Aranoa 🌹🔻 #GabilondoPresidente,2021-04-17 11:05:45+00:00,1383376129654812672,@ppmadrid @populares Con sobrecostes de 100 mi...,"[21, 88]",[Twitter for iPhone],0,0,3617,4997,151810,[],[],Partido Popular,mención
2,Toni🌾🥀,2021-04-17 11:05:44+00:00,1383376124365840385,Ventajas de esta estrategia:\n\n- los votantes...,"[0, 237]",[Twitter Web App],0,0,142,185,5079,[],[],Partido Popular,mención
3,Ángel Panizo Coleto,2021-04-17 11:05:42+00:00,1383376116166004747,@populares @IdiazAyuso Aqui están los datos de...,"[23, 300]",[Twitter for Android],0,0,433,127,4316,[],"[{'url': 'https://t.co/x1MBawrDo7', 'expanded_...",Partido Popular,mención
4,Salome,2021-04-17 11:05:39+00:00,1383376102983299076,@teresajbecerril @populares También hablabas a...,"[28, 104]",[Twitter for iPhone],0,0,200,381,3553,[],[],Partido Popular,mención
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Vox_Hellin,2021-04-17 10:58:55+00:00,1383374409545576455,‼️Nosotros luchamos por todos los españoles si...,"[0, 158]",[Twitter for Android],0,0,290,338,3340,"[{'text': 'ProtegeMadrid', 'indices': [125, 13...",[],Vox,mención
96,Francisco J.rey /🌹❤,2021-04-17 10:58:49+00:00,1383374384346267651,@Sirona33014070 @TaniaCrespo3 @LorenaCastro20 ...,"[60, 198]",[Twitter for Android],0,0,1005,991,5732,[],[],Vox,mención
97,christian9516,2021-04-17 10:58:40+00:00,1383374347344125957,@TabarniaB Sería noticia que Vox no llenará el...,"[11, 63]",[Twitter for Android],0,0,23,75,2859,[],[],Vox,mención
98,justi,2021-04-17 10:58:21+00:00,1383374264561061891,El doble pinchazo de Vox en Orcasitas: ni públ...,"[0, 87]",[Twitter for iPhone],0,0,954,976,81113,[],"[{'url': 'https://t.co/aNGIIqo431', 'expanded_...",Vox,mención
